In [22]:
# veterans history project dataset
import time
import requests
# import error type
from requests.exceptions import RequestException
# chucked encoding error
from requests.exceptions import ChunkedEncodingError
# data manipulation
import pandas as pd
# utilities
from helpers_loc import get_file_stats

In [23]:
# Constants for rate limiting and paging
RATE_LIMIT_DELAY = 3  # Delay in seconds between requests (20 requests per minute)
MAX_ITEMS_PER_PAGE = 1000  # Recommended maximum items per page
MAX_TOTAL_ITEMS = 100000  # Maximum items per query !! Set low for testing !!
RETRY_DELAY = 1  # Initial delay in seconds for retries
MAX_RETRIES = 5  # Maximum number of retries for rate-limited requests
DATA_URL_BASE = 'https://www.loc.gov/collections/veterans-history-project-collection'

In [4]:
# Function to parse the returned JSON
def fetch_url_with_rate_limit(rep_url, timeout=60):
    retries = 0
    # Retry logic with exponential backoff for 429 status codes
    while retries < MAX_RETRIES:
        try:
            print(f"Attempting to fetch URL: {rep_url}")
            response = requests.get(rep_url, timeout=timeout)
            if response.status_code == 429:  # Too Many Requests
                print("Rate limit exceeded. Pausing...")
                time.sleep(RETRY_DELAY * (2 ** retries))  # Exponential backoff
                retries += 1
                continue
            response.raise_for_status()  # Raise HTTPError for bad responses
            return response
        except requests.exceptions.RequestException as req_err:
            print(f"Request error occurred: {req_err}")
            retries += 1
            time.sleep(RETRY_DELAY * (2 ** retries))  # Exponential backoff
    raise Exception(f"Failed to fetch {rep_url} after {MAX_RETRIES} retries.")

In [4]:
# adding fo=json can return web representation: e.g. https://www.loc.gov/collections/veterans-history-project-collection?fo=json
web_rep_url = f'{DATA_URL_BASE}?fo=json&fa=online-format:online+text'
print(f"web representation url: {web_rep_url}")

web representation url: https://www.loc.gov/collections/veterans-history-project-collection?fo=json&fa=online-format:online+text


In [ ]:
# Initialize an empty list to store all results
all_results = []
# Loop through pages with rate limiting and paging limits
total_items_processed = 0
# Start with web_rep_url
current_url = web_rep_url
print("Start fetching information from each url:")
# MAX_TOTAL_ITEMS = 10
while current_url and total_items_processed < MAX_TOTAL_ITEMS:
    try:
        response = fetch_url_with_rate_limit(current_url)
        data = response.json()
        # Process the results on the current page
        results = data.get('results', [])
        all_results.extend(results)
        total_items_processed += len(results)
        print(f"Processed {len(results)} results. Total: {total_items_processed}")

        # Get the URL for the next page
        current_url = data.get('pagination', {}).get('next')
        print(f"Next page URL: {current_url}")

        # Delay to respect rate limits
        time.sleep(RATE_LIMIT_DELAY)
    except Exception as e:
        print(f"Error processing page {current_url}: {e}")
        break

# Convert the accumulated results into a DataFrame
if all_results:
    df_results = pd.DataFrame(all_results)
    print("DataFrame created successfully.")
else:
    print("No results found.")

Start fetching information from each url:
Attempting to fetch URL: https://www.loc.gov/collections/veterans-history-project-collection?fo=json&fa=online-format:online+text
Processed 25 results. Total: 25
Next page URL: https://www.loc.gov/collections/veterans-history-project-collection/?fa=online-format:online+text&fo=json&sp=2
DataFrame created successfully.


In [6]:
# print(f'Loaded web representation with {len(data):,} entries.')
len(df_results)

25

In [ ]:
# save df as parquet file
df_results.to_parquet('../data/raw/loc/veterans_history_project.parquet', index=False)

In [ ]:
# retrieve the df_results DataFrame from the parquet file
df_results = pd.read_parquet('../data/raw/loc/veterans_history_project.parquet')
print(df_results.info())
df_results.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3761 entries, 0 to 3760
Data columns (total 53 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   access_restricted              3761 non-null   bool  
 1   aka                            3761 non-null   object
 2   campaigns                      3761 non-null   object
 3   contributor                    3761 non-null   object
 4   contributor_interviewer        3731 non-null   object
 5   contributor_organization       3445 non-null   object
 6   contributor_veteran            3761 non-null   object
 7   date                           3585 non-null   object
 8   dates                          3585 non-null   object
 9   description                    3759 non-null   object
 10  digitized                      3761 non-null   bool  
 11  extract_timestamp              3761 non-null   object
 12  group                          3761 non-null   object
 13  has

,access_restricted,aka,campaigns,contributor,contributor_interviewer,contributor_organization,contributor_veteran,date,dates,description,...,timestamp,title,url,subject_race,number_pow,segments,partof_serving_our_voices,type,number_date_created_first_itm,year_record_created
0,False,"[http://www.loc.gov/item/afc2001001.00704/, fi...",[],"[schmidt, karen, cumbee, amanda, medina high s...","[cumbee, amanda]",[medina high school],"[zarney, elmer a.]",1943,[[1943 TO 1945]],"[Captain, Army Air Forces/Corps, World War, 19...",...,2025-10-19T01:04:09.896Z,Elmer A. Zarney Collection,https://www.loc.gov/item/afc2001001.00704/,None,None,None,None,None,2002-06-25,2002
1,False,"[http://www.loc.gov/item/afc2001001.00744/, fi...",[],"[schmidt, karen, costa, michelle, zambory, edw...","[costa, michelle]",[medina high school],"[zambory, edward]",1948,[[1948 TO 1952]],"[Sergeant, Air Force, Cold War - Laughlin Air ...",...,2025-10-19T01:04:55.576Z,Edward Zambory Collection,https://www.loc.gov/item/afc2001001.00744/,None,None,None,None,None,2002-06-26,2002
2,False,"[http://www.loc.gov/item/afc2001001.00315/, fi...",[],"[summit parkway middle school, addison, glenn ...","[white, charles christopher]",[summit parkway middle school],"[addison, glenn d.]",1979,[1979],"[Lieutenant Colonel, Air Force, Persian Gulf W...",...,2025-10-19T01:01:59.253Z,Glenn D. Addison Collection,https://www.loc.gov/item/afc2001001.00315/,None,None,None,None,None,2002-05-23,2002
3,False,"[http://www.loc.gov/item/afc2001001.01928/, fi...",[],"[phillips, harold b., stewart bell, jr. archiv...","[phillips, harold b.]","[stewart bell, jr. archives, handley regional ...","[zuckerman, joseph]",1941,[[1941 TO 1946]],"[Staff Sergeant, Army, World War, 1939-1945 - ...",...,2025-10-19T01:19:39.820Z,Joseph Zuckerman Collection,https://www.loc.gov/item/afc2001001.01928/,None,None,None,None,None,2002-10-01,2002
4,False,"[http://www.loc.gov/item/afc2001001.00372/, fi...",[],"[lugar, richard, dinger, dallas l., u.s. senat...","[sanders, timothy j.]",[u.s. senate/richard lugar],"[dinger, dallas l.]",1950,[[1950 TO 1951]],"[Private First Class, Army, Korean War, 1950-1...",...,2025-10-19T01:01:59.253Z,Dallas L. Dinger Collection,https://www.loc.gov/item/afc2001001.00372/,None,None,"[{'count': 2, 'link': 'https://www.loc.gov/col...",None,None,2002-05-29,2002


In [25]:
# for each column print the first row data
for col in df_results.columns:
    print(f'{col}: {df_results[col].iloc[0]}')

access_restricted: False
aka: ['http://www.loc.gov/item/afc2001001.00704/'
 'file:///natlib/afc2001001/registry/mets/00/loc.natlib.afc2001001.00704.xml'
 'https://tile.loc.gov/storage-services/natlib/afc2001001/vhp-stories/mets/loc.natlib.afc2001001.00704.xml'
 'http://www.loc.gov/resource/afc2001001.00704.sr0001001/']
campaigns: []
contributor: ['schmidt, karen' 'cumbee, amanda' 'medina high school' 'zarney, elmer a.']
contributor_interviewer: ['cumbee, amanda']
contributor_organization: ['medina high school']
contributor_veteran: ['zarney, elmer a.']
date: 1943
dates: ['[1943 TO 1945]']
description: ['Captain, Army Air Forces/Corps, World War, 1939-1945 - United States; European Theater.']
digitized: True
extract_timestamp: 2025-10-18T19:52:48.618Z
group: ['veterans-mets-00' 'veterans-mets']
hassegments: False
id: http://www.loc.gov/item/afc2001001.00704/
image_url: ['https://www.loc.gov/static/images/original-format/personal-narrative.svg']
index: 5449
item: {'birth_place': 'Clevela

In [ ]:
# reconstruct the dataframe where each row contains only one media resource
l_resource = []
for n in range(len(df_results)):
    collection_number = df_results['item'][n]['collection_number']
    title = df_results['title'][n]
    description = df_results['description'][n]
    dates = df_results['dates'][n]
    language = df_results['language'][n]
    location = df_results['location'][n]
    location_home = df_results['location_home'][n]
    location_service = df_results['location_service'][n]
    partof = df_results['partof'][n]
    subject = df_results['subject'][n]
    subject_battles = df_results['subject_battles'][n]
    subject_branch = df_results['subject_branch'][n]
    subject_conflict = df_results['subject_conflict'][n]
    subject_entrance = df_results['subject_entrance'][n]
    subject_format = df_results['subject_format'][n]
    subject_gender = df_results['subject_gender'][n]
    subject_rank = df_results['subject_rank'][n]
    subject_status = df_results['subject_status'][n]
    subject_unit = df_results['subject_unit'][n]
    subject_race = df_results['subject_race'][n]
    collection_resource = []

    for resource in df_results['resources'][n]:
        try:
            fulltext_file_url = resource.get('fulltext_file', None)
            retries = 0
            while retries < MAX_RETRIES:
                try:
                    if fulltext_file_url:
                        print(f"Fetching fulltext file: {fulltext_file_url}")
                        response = requests.get(fulltext_file_url, timeout=60)
                        if response.status_code == 429:  # Too Many Requests
                            print("Rate limit exceeded. Pausing...")
                            time.sleep(RETRY_DELAY * (2 ** retries))  # Exponential backoff
                            retries += 1
                            continue
                        response.raise_for_status()
                        fulltext_file_str = response.text
                        # print(f"Successfully fetched fulltext file: {fulltext_file_str}")
                    else:
                        fulltext_file_str = None
                    break
                except requests.exceptions.RequestException as req_err:
                    print(f"Request error occurred: {req_err}")
                    retries += 1
                    time.sleep(RETRY_DELAY * (2 ** retries))  # Exponential backoff
                except Exception as e:
                    print(f"Error fetching fulltext file: {e}")
                    fulltext_file_str = None
                    break
            video_url = resource.get('video', None)
            audio_url = resource.get('audio', None)
        except Exception as e:
            fulltext_file_url = None
            fulltext_file_str = None
            video_url = None
            audio_url = None
        finally:
            collection_resource.append({
            'collection_number': collection_number,
            'fulltext_file_url': fulltext_file_url,
            'fulltext_file_str': fulltext_file_str,
            'video_url': video_url,
            'audio_url': audio_url,
            'title': title,
            'description': description,
            'dates': dates,
            'language': language,
            'location': location,
            'location_home': location_home,
            'location_service': location_service,
            'partof': partof,
            'subject': subject,
            'subject_battles': subject_battles,
            'subject_branch': subject_branch,
            'subject_conflict': subject_conflict,
            'subject_entrance': subject_entrance,
            'subject_format': subject_format,
            'subject_gender': subject_gender,
            'subject_rank': subject_rank,
            'subject_status': subject_status,
            'subject_unit': subject_unit,
            'subject_race': subject_race       
            })
    l_resource.append(collection_resource)

In [6]:
#transform the list of resources into a DataFrame
df_resources = pd.DataFrame([item for sublist in l_resource for item in sublist])
df_resources.head()

,collection_number,fulltext_file_url,fulltext_file_str,video_url,audio_url,title,description,dates,language,location,...,subject_battles,subject_branch,subject_conflict,subject_entrance,subject_format,subject_gender,subject_rank,subject_status,subject_unit,subject_race
0,AFC/2001/001/704,None,None,None,None,Elmer A. Zarney Collection,"[Captain, Army Air Forces/Corps, World War, 19...",[[1943 TO 1945]],[english],"[ohio, united states, european theater]",...,None,[army air forces/corps],"[world war, 1939-1945]",None,[interview transcript],[male],"[captain, veteran]",[veteran],[111th tactical air command reconnaissance squ...,None
1,AFC/2001/001/744,None,None,None,None,Edward Zambory Collection,"[Sergeant, Air Force, Cold War - Laughlin Air ...",[[1948 TO 1952]],[english],"[alaska, laughlin air force base, texas, ohio,...",...,None,[air force],[cold war],[enlisted],[interview transcript],[male],"[sergeant, veteran]",[veteran],None,None
2,AFC/2001/001/315,None,None,None,None,Glenn D. Addison Collection,"[Lieutenant Colonel, Air Force, Persian Gulf W...",[1979],[english],"[prince sultan air base, saudi arabia, south c...",...,None,[air force],"[persian gulf war, 1991]",[enlisted],[interview transcript],[male],"[lieutenant colonel, veteran]",[veteran],"[169th fighter squadron, 4404th composite wing...",None
3,AFC/2001/001/1928,None,None,None,None,Joseph Zuckerman Collection,"[Staff Sergeant, Army, World War, 1939-1945 - ...",[[1941 TO 1946]],[english],"[virginia, european theater]",...,None,[army],"[world war, 1939-1945]",None,[interview transcript],[male],"[staff sergeant, veteran]",[veteran],None,None
4,AFC/2001/001/372,None,None,None,None,Dallas L. Dinger Collection,"[Private First Class, Army, Korean War, 1950-1...",[[1950 TO 1951]],[english],"[indiana, korea, camp atterbury, indiana]",...,None,[army],"[korean war, 1950-1953]",None,"[interview transcript, personal correspondence...",[male],"[private first class, veteran]",[veteran],"[5th infantry regiment, 28th infantry division...",None


In [11]:
len(df_resources)

8127

In [12]:
df_resources.columns

Index(['collection_number', 'fulltext_file_url', 'fulltext_file_str',
       'video_url', 'audio_url', 'title', 'description', 'dates', 'language',
       'location', 'location_home', 'location_service', 'partof', 'subject',
       'subject_battles', 'subject_branch', 'subject_conflict',
       'subject_entrance', 'subject_format', 'subject_gender', 'subject_rank',
       'subject_status', 'subject_unit', 'subject_race',
       'fulltext_file_str_cleaned', 'transcript_raw_text_only'],
      dtype='object')

In [8]:
import re
from bs4 import BeautifulSoup

def clean_raw_transcript_str(fulltext_file_str: str) -> str:
    l_transcript_lines = []
    # utilize bs4 xml parser
    soup = BeautifulSoup(fulltext_file_str, 'xml')
    # each sp tag in the document represents a "line" in the transcript
    for sp in soup.find_all('sp'):
        
        try:
            speaker = sp.find('speaker').get_text(strip=True)
        
        except:
            # placeholder speaker tag if not found
            speaker = "speaker_unknown"
        try:
            # return empty text if p tag not found
            spoken_text = sp.find('p').get_text(strip=True)
        
        except:
            spoken_text = ""
        
        l_transcript_lines.append(f"<{speaker}>{spoken_text}</{speaker}> ")
    
    # merge lines into one string
    transcript_lines = ''.join(l_transcript_lines)
    
    # remove (), [], {} and anything in between
    transcript_lines_stripped = re.sub(r'\([^)]*\)', '', transcript_lines)
    transcript_lines_stripped = re.sub(r'\[[^]]*\]', '', transcript_lines_stripped)
    transcript_lines_stripped = re.sub(r'\{[^}]*\)\}', '', transcript_lines_stripped)

    # remove double dashes and ellipsis
    transcript_lines_stripped = re.sub(r'--+', '', transcript_lines_stripped)
    transcript_lines_stripped = re.sub(r'\.{2,}', '', transcript_lines_stripped)

    # clean whitespace
    transcript_lines_stripped = re.sub(r'\s+', ' ', transcript_lines_stripped).strip()
    
    return transcript_lines_stripped

def remove_speaker_tag(transcript_lines_stripped: str) -> str:
    # remove <> and anything in between
    try:
        return re.sub(r'\<[^>]*\>', '', transcript_lines_stripped)
    except:
        return False
    
print("Extracting raw transcripts from dataframe")
df_resources['fulltext_file_str_cleaned'] = df_resources['fulltext_file_str'].astype(str).apply(clean_raw_transcript_str)
df_resources['transcript_raw_text_only'] = df_resources['fulltext_file_str_cleaned'].apply(remove_speaker_tag)

Extracting raw transcripts from dataframe


In [ ]:
# save the DataFrame to a parquet file
df_resources.to_parquet('../data/raw/loc/veterans_history_project_resources.parquet', index=False)

In [ ]:
# retrieve resource DataFrame from the parquet file
df_resources = pd.read_parquet('../data/raw/loc/veterans_history_project_resources.parquet')

In [ ]:
# !!!run this cell on local only!!!
# function to download audio/ video files from URLs, and store them under veteran_interviews/{idx}
def download_media_files(df):
    import os
    base_dir = '/Volumes/KINGSTON/veteran_interviews'
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)
    
    for idx, row in df.iterrows():
        # Prefer video if available, otherwise use audio
        media_type = None
        media_url = None
        if pd.notnull(row.get('video_url')) and row['video_url']:
            media_type = 'video'
            media_url = row['video_url']
            ext = '.mp4'
        elif pd.notnull(row.get('audio_url')) and row['audio_url']:
            media_type = 'audio'
            media_url = row['audio_url']
            ext = '.mp3'
        else:
            print(f"No audio or video URL for index {idx}")
            continue

        try:
            response = requests.get(media_url, timeout=60)
            response.raise_for_status()
            # Create a subdirectory for each index
            sub_dir = os.path.join(base_dir, str(idx))
            if not os.path.exists(sub_dir):
                os.makedirs(sub_dir)
            # skip downloading if the file already exists
            elif os.path.exists(os.path.join(sub_dir, f'{media_type}{ext}')):
                print(f"File already exists for index {idx}: {media_type}{ext}")
                continue
            else:
                file_path = os.path.join(sub_dir, f'{media_type}{ext}')
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                print(f"Downloaded {media_type} file for index {idx} to {file_path}")
        except Exception as e:
            print(f"Failed to download {media_type} file for index {idx}: {e}")
            
download_media_files(df_resources)

In [ ]:
# # helper: create splits for df_resources (average size to run processing)
# def create_splits(df, split_size=1000):
#     splits = []
#     for i in range(0, len(df), split_size):
#         splits.append(df.iloc[i:i + split_size])
#     return splits
# # Create splits of the DataFrame
# splits = create_splits(df_resources, split_size=1000)

In [ ]:
# helper to inspect raw transcript text structure
# import pprint
# pprint.pprint(df_resources['fulltext_file_str'].iloc[0])

In [ ]:
transcript_gt_sample = df_resources['transcript_raw_text_only'][1]
from pprint import pprint
pprint(transcript_gt_sample)

In [ ]:
transcript_gt_sample_120 = """
H. Marie Thomas. I'm speaking with what's your name? John Aaron, Jr. John Aaron, Jr. And you got your information on here, your birth date. 
I've got your address and information. 
I need the birth date. . Okay. August. And where were you born, city of birthplace? Little Rock, Arkansas All right. Okay.
What branch of service were you in? 
I was in the Navy first. You said first? Yes. I was in the Navy from 1941 to 1945. 
Then I went in the Reserve in I went back to school, and then in 1957 I received a commission, second lieutenant in the Army. 
Was that your highest ranking? I retired as a major. Oh, okay. Okay. 
Do you remember your serial number? Which one? You know, your social security number is this this is something you don't give out. Is this  No. It says serial number for service. 
It's different than social security number. Okay. That's fine. What battalion, regiment or division were you in? 
I was in the Navy. The first I was in the Navy.
"""